# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage30yr""","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage30yr""","""Unknown""",669,"""Laurianneshire""","""AutoLoans""","""Accepted"""
"""MoneyMarketSavingsAccount""","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmieshire""","""DepositAccounts""","""Accepted"""
"""UPlusFinPersonal""","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccounts""","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""8721744748880341658""","""17780134115081472705""",1.1297e19,"""../../../../data/SampleHDS.jso…","""17727080977590953557""","""4720659435396410217""",false
"""17657506305816802489""","""12602925573573530865""",4.9467e18,"""../../../../data/SampleHDS.jso…","""17727080977590953557""","""16354395086082494650""",true
"""8495700091119640092""","""570737860946070418""",1.7771e19,"""../../../../data/SampleHDS.jso…","""1493575742660549041""","""4720659435396410217""",false
"""2935824442795030428""","""12602925573573530865""",1.4654e18,"""../../../../data/SampleHDS.jso…","""1871987113938817970""","""4720659435396410217""",false
"""672269816166370467""","""17780134115081472705""",1.5969e19,"""../../../../data/SampleHDS.jso…","""1871987113938817970""","""1124735200720621161""",true
"""6370506178349658990""","""570737860946070418""",4.8177e18,"""../../../../data/SampleHDS.jso…","""9776856669484683721""",null,false
"""4655843819874918207""","""570737860946070418""",1.1208e19,"""../../../../data/SampleHDS.jso…","""1871987113938817970""","""1124735200720621161""",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'Customer_City': 'PREDICTOR_0',
 'Customer_MaritalStatus': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'filename': 'filename',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""1989194689959674902""","""12748091462306506616""",1.5307e19,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""10416962998088695094""","""Rejected"""
"""2255014448522950249""","""6967934381435303917""",1.5121e19,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""16612638657365535332""","""Accepted"""
"""6513904903080125239""","""3033157655102799613""",1.8219e19,"""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""10416962998088695094""","""Rejected"""
"""2750496242488270454""","""6967934381435303917""",1.5180e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""10416962998088695094""","""Rejected"""
"""1429658872754480146""","""12748091462306506616""",3.4904e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""622070092991234640""","""Accepted"""
"""167952866498927685""","""3033157655102799613""",7.4948e18,"""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""18091338816821033542""","""3033157655102799613""",1.8780e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""622070092991234640""","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,filename,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""16929868650087711875""","""4859399229005635569""",1.6319e19,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""2534513497512586427""","""Rejected"""
"""14247178193315051957""","""12681019335483465754""",1.5122e19,"""../../../../data/SampleHDS.jso…","""FirstMortgage30yr""","""14117068583901799656""","""Accepted"""
"""5609459030902291423""","""12638301940606791516""",4.8193e18,"""../../../../data/SampleHDS.jso…","""MoneyMarketSavingsAccount""","""2534513497512586427""","""Rejected"""
"""10801425033456365729""","""12681019335483465754""",1.6879e19,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""2534513497512586427""","""Rejected"""
"""11248097320504005415""","""4859399229005635569""",1.5298e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""5180137805114566263""","""Accepted"""
"""3171898946285664141""","""12638301940606791516""",1.3198e19,"""../../../../data/SampleHDS.jso…","""UPlusFinPersonal""",null,"""Rejected"""
"""1117822464583473178""","""12638301940606791516""",3.5878e18,"""../../../../data/SampleHDS.jso…","""BasicChecking""","""5180137805114566263""","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

Customer_City=PREDICTOR_0
Customer_MaritalStatus=PREDICTOR_1
Customer_CLV=PREDICTOR_2
filename=filename
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_0,PREDICTOR_1,PREDICTOR_2,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str
"""17370129273884146750""","""8364273036859430566""",1.0389e19,"""FirstMortgage30yr""","""13316372397012808074""","""Rejected"""
"""17175413995010399158""","""4128535783622229334""",7.4165e18,"""FirstMortgage30yr""","""15147807493915068473""","""Accepted"""
"""7747391926491757433""","""3482510297199845548""",4.1077e18,"""MoneyMarketSavingsAccount""","""13316372397012808074""","""Rejected"""
"""3974640675738568972""","""4128535783622229334""",5.7397e18,"""BasicChecking""","""13316372397012808074""","""Rejected"""
"""15560386373563484816""","""8364273036859430566""",1.1162e19,"""BasicChecking""","""11244957752214382544""","""Accepted"""
"""5671329845244880936""","""3482510297199845548""",2.4774e18,"""UPlusFinPersonal""",null,"""Rejected"""
"""10994670432275401250""","""3482510297199845548""",1.6249e19,"""BasicChecking""","""11244957752214382544""","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.

sys:1: MapWithoutReturnDtypeWarning:

Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.



ComputeError: AttributeError: 'int' object has no attribute 'encode'